# Train you first forecasting model

So you picked the forecasting project. Good for you! Forecasting is an important skill, and it's helpful to learn about the techchnologies out there that can make your life easier. Or worse, depending on how you look at it. ;)

### 1. Download the Kaggle forecasting data
Use the notebook example for this.

### 2. Access your data

In [1]:
import pandas as pd
import json
import sagemaker
import boto3

In [2]:
df = pd.read_csv('train.csv')

In [3]:
df.shape

(5113, 99)

In [4]:
df.head()

,Date,ACME,ADAX,ALTU,APAC,ARNE,BEAV,BESS,BIXB,BLAC,...,VINI,WASH,WATO,WAUR,WEAT,WEST,WILB,WIST,WOOD,WYNO
0,19940101,12384900,11930700,12116700,12301200,10706100,10116900,11487900,11182800,10848300,...,10771800,12116400,11308800,12361800,11331600,10644300,11715600,11241000,10490100,10545300
1,19940102,11908500,9778500,10862700,11666400,8062500,9262800,9235200,3963300,3318300,...,4314300,10733400,9154800,12041400,9168300,4082700,9228000,5829900,7412100,3345300
2,19940103,12470700,9771900,12627300,12782700,11618400,10789800,11895900,4512600,5266500,...,2976900,11775000,10700400,12687300,11324400,2746500,3686700,4488900,9712200,4442100
3,19940104,12725400,6466800,13065300,12817500,12134400,11816700,12186600,3212700,8270100,...,3476400,12159600,11907000,12953100,11903700,2741400,4905000,4089300,11401500,4365000
4,19940105,10894800,11545200,8060400,10379400,6918600,9936300,6411300,9566100,8009400,...,6393300,11419500,7334400,10178700,7471500,8235300,11159100,10651500,10006200,8568300


Each row here is a new point in time, and each column is an energy station. That means that each COLUMN is a unique time series data set. We are going to train our first model on a single column. Then, you can extend it by adding more columns.

In [5]:
# build list of locations
columns = list(df.columns[1:])
columns = columns
# columns[:85]
print(columns)

['ACME', 'ADAX', 'ALTU', 'APAC', 'ARNE', 'BEAV', 'BESS', 'BIXB', 'BLAC', 'BOIS', 'BOWL', 'BREC', 'BRIS', 'BUFF', 'BURB', 'BURN', 'BUTL', 'BYAR', 'CAMA', 'CENT', 'CHAN', 'CHER', 'CHEY', 'CHIC', 'CLAY', 'CLOU', 'COOK', 'COPA', 'DURA', 'ELRE', 'ERIC', 'EUFA', 'FAIR', 'FORA', 'FREE', 'FTCB', 'GOOD', 'GUTH', 'HASK', 'HINT', 'HOBA', 'HOLL', 'HOOK', 'HUGO', 'IDAB', 'JAYX', 'KENT', 'KETC', 'LAHO', 'LANE', 'MADI', 'MANG', 'MARE', 'MAYR', 'MCAL', 'MEDF', 'MEDI', 'MIAM', 'MINC', 'MTHE', 'NEWK', 'NINN', 'NOWA', 'OILT', 'OKEM', 'OKMU', 'PAUL', 'PAWN', 'PERK', 'PRYO', 'PUTN', 'REDR', 'RETR', 'RING', 'SALL', 'SEIL', 'SHAW', 'SKIA', 'SLAP', 'SPEN', 'STIG', 'STIL', 'STUA', 'SULP', 'TAHL', 'TALI', 'TIPT', 'TISH', 'VINI', 'WASH', 'WATO', 'WAUR', 'WEAT', 'WEST', 'WILB', 'WIST', 'WOOD', 'WYNO']


In [6]:
target = df['ACME']

In [7]:
target.shape

(5113,)

Great! We have 5113 obervations. That's well over the 300-limit on DeepAR.

### 3. Create Train and Test Sets
Now, we'll build 2 datasets. One for training, another for testing. Both need to be written to json files, then copied over to S3.

In [43]:
def get_split(df, freq='D', split_type = 'train', cols_to_use = ['ACME']):
    rt_set = []
    
    # use 70% for training
    if split_type == 'train':
        lower_bound = 0
        upper_bound = round(df.shape[0] * .7)
        
    # use 15% for validation
    elif split_type == 'validation':
        lower_bound = round(df.shape[0] * .7)
        upper_bound = round(df.shape[0] * .85)
        
    # use 15% for test
    elif split_type == 'test':
        lower_bound = round(df.shape[0] * .85)
        upper_bound = df.shape[0]
            
    # loop through columns you want to use
    for h in list(df):
        if h in cols_to_use:
            
            target_column = df[h].values.tolist()[lower_bound:upper_bound]
            
            date_str = str(df.iloc[0]['Date'])
            
            year = date_str[0:4]
            month = date_str[4:6]
            date = date_str[7:]
                                                
            start_dataset = pd.Timestamp("{}-{}-{} 00:00:00".format(year, month, date, freq=freq))
                        
            # create a new json object for each column
            json_obj = {
                        'start': str(start_dataset),
                        'target': target_column,
                        'station': h
                       }
    
            rt_set.append(json_obj)
    
    return rt_set

In [44]:
train_set = get_split(df, cols_to_use = columns)
test_set = get_split(df, split_type = 'test', cols_to_use = columns)

In [45]:
def write_dicts_to_file(path, data):
    with open(path, 'wb') as fp:
        for d in data:
            # fp.write(json.dumps(data).encode("utf-8"))
            fp.write(json.dumps(d).encode("utf-8"))
            fp.write("\n".encode('utf-8'))

In [46]:
write_dicts_to_file('train.json', train_set)
write_dicts_to_file('test.json', test_set)

In [47]:
!aws s3 cp train.json s3://forecasting-do-not-delete/train/train.json
!aws s3 cp test.json s3://forecasting-do-not-delete/test/test.json

upload: ./train.json to s3://forecasting-do-not-delete/train/train.json
upload: ./test.json to s3://forecasting-do-not-delete/test/test.json


### 4. Run a SageMaker Training Job
Ok! If everything worked, we should be able to train a model in SageMaker straight away.

In [48]:
sess = sagemaker.Session()
region = sess.boto_region_name
image = sagemaker.amazon.amazon_estimator.get_image_uri(region, "forecasting-deepar", "latest")
role = sagemaker.get_execution_role()   

In [49]:
estimator = sagemaker.estimator.Estimator(
    sagemaker_session=sess,
    image_name=image,
    role=role,
    train_instance_count=1,
    train_instance_type='ml.c4.2xlarge',
    base_job_name='deepar-electricity-demo',
    output_path='s3://forecasting-do-not-delete/output'
)

In [50]:
hyperparameters = {
    
    # frequency interval is once per day
    "time_freq": 'D',
    "epochs": "400",
    "early_stopping_patience": "40",
    "mini_batch_size": "64",
    "learning_rate": "5E-4",
    
    # let's use the last 30 days for context
    "context_length": str(30),
    
    # let's forecast for 30 days
    "prediction_length": str(30)
}

estimator.set_hyperparameters(**hyperparameters)

In [51]:
data_channels = {
    "train": "s3://forecasting-do-not-delete/train/train.json",
    "test": "s3://forecasting-do-not-delete/test/test.json"
}

estimator.fit(inputs=data_channels, wait=True)

2019-08-01 21:47:28 Starting - Starting the training job...
2019-08-01 21:47:30 Starting - Launching requested ML instances......
2019-08-01 21:48:35 Starting - Preparing the instances for training...
2019-08-01 21:49:29 Downloading - Downloading input data......
2019-08-01 21:50:20 Training - Training image download completed. Training in progress.
Arguments: train
[08/01/2019 21:50:22 INFO 140504085288768] Reading default configuration from /opt/amazon/lib/python2.7/site-packages/algorithm/resources/default-input.json: {u'num_dynamic_feat': u'auto', u'dropout_rate': u'0.10', u'mini_batch_size': u'128', u'test_quantiles': u'[0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]', u'_tuning_objective_metric': u'', u'_num_gpus': u'auto', u'num_eval_samples': u'100', u'learning_rate': u'0.001', u'num_cells': u'40', u'num_layers': u'2', u'embedding_dimension': u'10', u'_kvstore': u'auto', u'_num_kv_servers': u'auto', u'cardinality': u'auto', u'likelihood': u'student-t', u'early_stopping_patience':

[08/01/2019 21:50:35 INFO 140504085288768] Epoch[11] Batch[0] avg_epoch_loss=17.102144
[08/01/2019 21:50:35 INFO 140504085288768] #quality_metric: host=algo-1, epoch=11, batch=0 train loss <loss>=17.1021442413
[08/01/2019 21:50:36 INFO 140504085288768] Epoch[11] Batch[5] avg_epoch_loss=17.078759
[08/01/2019 21:50:36 INFO 140504085288768] #quality_metric: host=algo-1, epoch=11, batch=5 train loss <loss>=17.0787588755
[08/01/2019 21:50:36 INFO 140504085288768] Epoch[11] Batch [5]#011Speed: 918.15 samples/sec#011loss=17.078759
[08/01/2019 21:50:36 INFO 140504085288768] processed a total of 566 examples
#metrics {"Metrics": {"update.time": {"count": 1, "max": 994.5809841156006, "sum": 994.5809841156006, "min": 994.5809841156006}}, "EndTime": 1564696236.418934, "Dimensions": {"Host": "algo-1", "Operation": "training", "Algorithm": "AWS/DeepAR"}, "StartTime": 1564696235.424285}

[08/01/2019 21:50:36 INFO 140504085288768] #throughput_metric: host=algo-1, train throughput=569.016354951 records

[08/01/2019 21:50:45 INFO 140504085288768] processed a total of 604 examples
#metrics {"Metrics": {"update.time": {"count": 1, "max": 1112.2829914093018, "sum": 1112.2829914093018, "min": 1112.2829914093018}}, "EndTime": 1564696245.849728, "Dimensions": {"Host": "algo-1", "Operation": "training", "Algorithm": "AWS/DeepAR"}, "StartTime": 1564696244.737006}

[08/01/2019 21:50:45 INFO 140504085288768] #throughput_metric: host=algo-1, train throughput=542.969662314 records/second
[08/01/2019 21:50:45 INFO 140504085288768] #progress_metric: host=algo-1, completed 5 % of epochs
[08/01/2019 21:50:45 INFO 140504085288768] #quality_metric: host=algo-1, epoch=20, train loss <loss>=16.9834011078
[08/01/2019 21:50:45 INFO 140504085288768] loss did not improve
[08/01/2019 21:50:46 INFO 140504085288768] Epoch[21] Batch[0] avg_epoch_loss=16.998318
[08/01/2019 21:50:46 INFO 140504085288768] #quality_metric: host=algo-1, epoch=21, batch=0 train loss <loss>=16.9983177185
[08/01/2019 21:50:46 INFO 140504

[08/01/2019 21:50:55 INFO 140504085288768] Epoch[30] Batch[0] avg_epoch_loss=16.833092
[08/01/2019 21:50:55 INFO 140504085288768] #quality_metric: host=algo-1, epoch=30, batch=0 train loss <loss>=16.8330917358
[08/01/2019 21:50:56 INFO 140504085288768] Epoch[30] Batch[5] avg_epoch_loss=16.908401
[08/01/2019 21:50:56 INFO 140504085288768] #quality_metric: host=algo-1, epoch=30, batch=5 train loss <loss>=16.9084008535
[08/01/2019 21:50:56 INFO 140504085288768] Epoch[30] Batch [5]#011Speed: 917.19 samples/sec#011loss=16.908401
[08/01/2019 21:50:56 INFO 140504085288768] processed a total of 560 examples
#metrics {"Metrics": {"update.time": {"count": 1, "max": 995.6960678100586, "sum": 995.6960678100586, "min": 995.6960678100586}}, "EndTime": 1564696256.414536, "Dimensions": {"Host": "algo-1", "Operation": "training", "Algorithm": "AWS/DeepAR"}, "StartTime": 1564696255.418407}

[08/01/2019 21:50:56 INFO 140504085288768] #throughput_metric: host=algo-1, train throughput=562.356254773 records

[08/01/2019 21:51:05 INFO 140504085288768] processed a total of 585 examples
#metrics {"Metrics": {"update.time": {"count": 1, "max": 1042.0000553131104, "sum": 1042.0000553131104, "min": 1042.0000553131104}}, "EndTime": 1564696265.753263, "Dimensions": {"Host": "algo-1", "Operation": "training", "Algorithm": "AWS/DeepAR"}, "StartTime": 1564696264.71121}

[08/01/2019 21:51:05 INFO 140504085288768] #throughput_metric: host=algo-1, train throughput=561.366497029 records/second
[08/01/2019 21:51:05 INFO 140504085288768] #progress_metric: host=algo-1, completed 10 % of epochs
[08/01/2019 21:51:05 INFO 140504085288768] #quality_metric: host=algo-1, epoch=39, train loss <loss>=16.8461891174
[08/01/2019 21:51:05 INFO 140504085288768] best epoch loss so far
[08/01/2019 21:51:05 INFO 140504085288768] Saved checkpoint to "/opt/ml/model/state_2315d263-b5a8-461a-a5cc-af5353c5005a-0000.params"
#metrics {"Metrics": {"state.serialize.time": {"count": 1, "max": 21.745920181274414, "sum": 21.7459201812

...

[08/01/2019 21:53:15 INFO 140504085288768] Epoch[164] Batch[5] avg_epoch_loss=16.770687
[08/01/2019 21:53:15 INFO 140504085288768] #quality_metric: host=algo-1, epoch=164, batch=5 train loss <loss>=16.7706871033
[08/01/2019 21:53:15 INFO 140504085288768] Epoch[164] Batch [5]#011Speed: 952.23 samples/sec#011loss=16.770687
[08/01/2019 21:53:16 INFO 140504085288768] processed a total of 611 examples
#metrics {"Metrics": {"update.time": {"count": 1, "max": 1027.3947715759277, "sum": 1027.3947715759277, "min": 1027.3947715759277}}, "EndTime": 1564696396.058436, "Dimensions": {"Host": "algo-1", "Operation": "training", "Algorithm": "AWS/DeepAR"}, "StartTime": 1564696395.030557}

[08/01/2019 21:53:16 INFO 140504085288768] #throughput_metric: host=algo-1, train throughput=594.655254596 records/second
[08/01/2019 21:53:16 INFO 140504085288768] #progress_metric: host=algo-1, completed 41 % of epochs
[08/01/2019 21:53:16 INFO 140504085288768] #quality_metric: host=algo-1, epoch=164, train loss <l

[08/01/2019 21:53:26 INFO 140504085288768] Epoch[174] Batch[5] avg_epoch_loss=16.773691
[08/01/2019 21:53:26 INFO 140504085288768] #quality_metric: host=algo-1, epoch=174, batch=5 train loss <loss>=16.7736908595
[08/01/2019 21:53:26 INFO 140504085288768] Epoch[174] Batch [5]#011Speed: 934.19 samples/sec#011loss=16.773691
[08/01/2019 21:53:26 INFO 140504085288768] processed a total of 596 examples
#metrics {"Metrics": {"update.time": {"count": 1, "max": 1076.362133026123, "sum": 1076.362133026123, "min": 1076.362133026123}}, "EndTime": 1564696406.369737, "Dimensions": {"Host": "algo-1", "Operation": "training", "Algorithm": "AWS/DeepAR"}, "StartTime": 1564696405.292925}

[08/01/2019 21:53:26 INFO 140504085288768] #throughput_metric: host=algo-1, train throughput=553.656897331 records/second
[08/01/2019 21:53:26 INFO 140504085288768] #progress_metric: host=algo-1, completed 43 % of epochs
[08/01/2019 21:53:26 INFO 140504085288768] #quality_metric: host=algo-1, epoch=174, train loss <loss

[08/01/2019 21:53:35 INFO 140504085288768] Epoch[184] Batch[0] avg_epoch_loss=16.781076
[08/01/2019 21:53:35 INFO 140504085288768] #quality_metric: host=algo-1, epoch=184, batch=0 train loss <loss>=16.7810764313
[08/01/2019 21:53:36 INFO 140504085288768] Epoch[184] Batch[5] avg_epoch_loss=16.714217
[08/01/2019 21:53:36 INFO 140504085288768] #quality_metric: host=algo-1, epoch=184, batch=5 train loss <loss>=16.714217186
[08/01/2019 21:53:36 INFO 140504085288768] Epoch[184] Batch [5]#011Speed: 927.79 samples/sec#011loss=16.714217
[08/01/2019 21:53:36 INFO 140504085288768] processed a total of 565 examples
#metrics {"Metrics": {"update.time": {"count": 1, "max": 972.0330238342285, "sum": 972.0330238342285, "min": 972.0330238342285}}, "EndTime": 1564696416.441211, "Dimensions": {"Host": "algo-1", "Operation": "training", "Algorithm": "AWS/DeepAR"}, "StartTime": 1564696415.468708}

[08/01/2019 21:53:36 INFO 140504085288768] #throughput_metric: host=algo-1, train throughput=581.1878317 recor

[08/01/2019 21:53:45 INFO 140504085288768] Epoch[193] Batch[5] avg_epoch_loss=16.740213
[08/01/2019 21:53:45 INFO 140504085288768] #quality_metric: host=algo-1, epoch=193, batch=5 train loss <loss>=16.7402127584
[08/01/2019 21:53:45 INFO 140504085288768] Epoch[193] Batch [5]#011Speed: 957.89 samples/sec#011loss=16.740213
[08/01/2019 21:53:46 INFO 140504085288768] processed a total of 633 examples
#metrics {"Metrics": {"update.time": {"count": 1, "max": 1060.379981994629, "sum": 1060.379981994629, "min": 1060.379981994629}}, "EndTime": 1564696426.108314, "Dimensions": {"Host": "algo-1", "Operation": "training", "Algorithm": "AWS/DeepAR"}, "StartTime": 1564696425.047476}

[08/01/2019 21:53:46 INFO 140504085288768] #throughput_metric: host=algo-1, train throughput=596.889923151 records/second
[08/01/2019 21:53:46 INFO 140504085288768] #progress_metric: host=algo-1, completed 48 % of epochs
[08/01/2019 21:53:46 INFO 140504085288768] #quality_metric: host=algo-1, epoch=193, train loss <loss


2019-08-01 21:54:03 Uploading - Uploading generated training model
2019-08-01 21:54:03 Completed - Training job completed
Billable seconds: 275


# 5. Run Inference
If you made it this far, congratulations! None of this is easy. For your next steps, please open up the example notebook under the SageMakerExamples:

- SageMakerExamples/Introduction To Amazon Algorithms/DeepAR-Electricity.
- **Guys I found this online - not sure if this is the right thing or not but wanted to add it here in case we don't get to it before I have to leave**
    - https://github.com/awslabs/amazon-sagemaker-examples/blob/master/introduction_to_amazon_algorithms/deepar_electricity/DeepAR-Electricity.ipynb

That will walk you through both how to add more timeseries to your model, and how to get inference results out of it.

In [20]:
class DeepARPredictor(sagemaker.predictor.RealTimePredictor):
    
    def __init__(self, *args, **kwargs):
        super().__init__(*args, content_type=sagemaker.content_types.CONTENT_TYPE_JSON, **kwargs)
        
    def predict(self, ts, cat=None, dynamic_feat=None, 
                num_samples=100, return_samples=False, quantiles=["0.1", "0.5", "0.9"]):
        """Requests the prediction of for the time series listed in `ts`, each with the (optional)
        corresponding category listed in `cat`.
        
        ts -- `pandas.Series` object, the time series to predict
        cat -- integer, the group associated to the time series (default: None)
        num_samples -- integer, number of samples to compute at prediction time (default: 100)
        return_samples -- boolean indicating whether to include samples in the response (default: False)
        quantiles -- list of strings specifying the quantiles to compute (default: ["0.1", "0.5", "0.9"])
        
        Return value: list of `pandas.DataFrame` objects, each containing the predictions
        """
        prediction_time = ts.index[-1] + 1
        quantiles = [str(q) for q in quantiles]
        req = self.__encode_request(ts, cat, dynamic_feat, num_samples, return_samples, quantiles)
        res = super(DeepARPredictor, self).predict(req)
        return self.__decode_response(res, ts.index.freq, prediction_time, return_samples)
    
    def __encode_request(self, ts, cat, dynamic_feat, num_samples, return_samples, quantiles):
        instance = series_to_dict(ts, cat if cat is not None else None, dynamic_feat if dynamic_feat else None)

        configuration = {
            "num_samples": num_samples,
            "output_types": ["quantiles", "samples"] if return_samples else ["quantiles"],
            "quantiles": quantiles
        }
        
        http_request_data = {
            "instances": [instance],
            "configuration": configuration
        }
        
        return json.dumps(http_request_data).encode('utf-8')
    
    def __decode_response(self, response, freq, prediction_time, return_samples):
        # we only sent one time series so we only receive one in return
        # however, if possible one will pass multiple time series as predictions will then be faster
        predictions = json.loads(response.decode('utf-8'))['predictions'][0]
        prediction_length = len(next(iter(predictions['quantiles'].values())))
        prediction_index = pd.DatetimeIndex(start=prediction_time, freq=freq, periods=prediction_length)        
        if return_samples:
            dict_of_samples = {'sample_' + str(i): s for i, s in enumerate(predictions['samples'])}
        else:
            dict_of_samples = {}
        return pd.DataFrame(data={**predictions['quantiles'], **dict_of_samples}, index=prediction_index)

    def set_frequency(self, freq):
        self.freq = freq
        
def encode_target(ts):
    return [x if np.isfinite(x) else "NaN" for x in ts]        

def series_to_dict(ts, cat=None, dynamic_feat=None):
    """Given a pandas.Series object, returns a dictionary encoding the time series.

    ts -- a pands.Series object with the target time series
    cat -- an integer indicating the time series category

    Return value: a dictionary
    """
    obj = {"start": str(ts.index[0]), "target": encode_target(ts)}
    if cat is not None:
        obj["cat"] = cat
    if dynamic_feat is not None:
        obj["dynamic_feat"] = dynamic_feat        
    return obj

In [21]:
predictor = estimator.deploy(
    initial_instance_count=1,
    instance_type='ml.m4.xlarge',
    predictor_cls=DeepARPredictor)

Using already existing model: deepar-electricity-demo-2019-08-01-19-36-37-058


ClientError: An error occurred (ValidationException) when calling the CreateEndpoint operation: Cannot create already existing endpoint "arn:aws:sagemaker:us-east-1:023375022819:endpoint/deepar-electricity-demo-2019-08-01-19-36-37-058".

In [22]:
predictor.predict(ts=timeseries[120], quantiles=[0.10, 0.5, 0.90]).head()

NameError: name 'timeseries' is not defined

In [27]:
num_timeseries = df.shape[1]
dir(df)
data_kw = df.resample('1D').sum()
print(data_kw)
# # timeseries = []
# for i in range(num_timeseries):
#     timeseries.append(np.trim_zeros(data_kw.iloc[:,i], trim='f'))

['ACME',
 'ADAX',
 'ALTU',
 'APAC',
 'ARNE',
 'BEAV',
 'BESS',
 'BIXB',
 'BLAC',
 'BOIS',
 'BOWL',
 'BREC',
 'BRIS',
 'BUFF',
 'BURB',
 'BURN',
 'BUTL',
 'BYAR',
 'CAMA',
 'CENT',
 'CHAN',
 'CHER',
 'CHEY',
 'CHIC',
 'CLAY',
 'CLOU',
 'COOK',
 'COPA',
 'DURA',
 'Date',
 'ELRE',
 'ERIC',
 'EUFA',
 'FAIR',
 'FORA',
 'FREE',
 'FTCB',
 'GOOD',
 'GUTH',
 'HASK',
 'HINT',
 'HOBA',
 'HOLL',
 'HOOK',
 'HUGO',
 'IDAB',
 'JAYX',
 'KENT',
 'KETC',
 'LAHO',
 'LANE',
 'MADI',
 'MANG',
 'MARE',
 'MAYR',
 'MCAL',
 'MEDF',
 'MEDI',
 'MIAM',
 'MINC',
 'MTHE',
 'NEWK',
 'NINN',
 'NOWA',
 'OILT',
 'OKEM',
 'OKMU',
 'PAUL',
 'PAWN',
 'PERK',
 'PRYO',
 'PUTN',
 'REDR',
 'RETR',
 'RING',
 'SALL',
 'SEIL',
 'SHAW',
 'SKIA',
 'SLAP',
 'SPEN',
 'STIG',
 'STIL',
 'STUA',
 'SULP',
 'T',
 'TAHL',
 'TALI',
 'TIPT',
 'TISH',
 'VINI',
 'WASH',
 'WATO',
 'WAUR',
 'WEAT',
 'WEST',
 'WILB',
 'WIST',
 'WOOD',
 'WYNO',
 '_AXIS_ALIASES',
 '_AXIS_IALIASES',
 '_AXIS_LEN',
 '_AXIS_NAMES',
 '_AXIS_NUMBERS',
 '_AXIS_ORDERS',
 

### 6. Extend Your Solution
Now you're getting forecasts, how will you extend your solution? How good are your forecasts? What about getting forecasts for the other stations? Is your model cognizant of the weather?

Spend your remaining time growing your modeling solution to leverage additional datasets. Then, think through how you'd set this up to run in production.